<a href="https://colab.research.google.com/github/KhalidSwaid/CloudCourse/blob/main/lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install firebase

In [ ]:

!pip install nltk
import nltk
from nltk.chat.util import Chat, reflections

In [ ]:

cloud_explanation = """Private Cloud: A private cloud is a cloud computing environment that is dedicated solely to one organization.
It is usually managed either by the organization itself or by a third-party service provider. The infrastructure and services are not shared
with other organizations, offering greater control, security, and customization options. Private clouds are typically used by large enterprises
or organizations with specific compliance or security requirements.
Public Cloud: A public cloud is a cloud computing environment that is hosted and managed by a third-party service provider and made available
to multiple organizations or individuals over the internet. The infrastructure and resources are shared among multiple users,
 offering scalability, flexibility, and cost-effectiveness. Public clouds are accessible to anyone who wants to use them and are often used
 for general-purpose computing needs such as web hosting, development, and testing.
Hybrid Cloud: A hybrid cloud is a computing environment that combines elements of both private and public clouds. It allows organizations
 to leverage the benefits of both environments by integrating on-premises infrastructure with public cloud resources. This integration enables
 data and applications to be dynamically moved between the two environments based on changing workload demands, security requirements, and cost
 considerations. Hybrid clouds offer greater flexibility, scalability, and customization options compared to using either private or public clouds alone.
 They are commonly used by organizations that want to retain control over sensitive data while taking advantage of the scalability and cost-effectiveness of
 public cloud services."""

In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Define some patterns and responses
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me ChatGPT.', 'I go by the name ChatGPT.']),
    (r'your name (.*)', ['You can call me ChatGPT.', 'I go by the name ChatGPT.']),
    (r'give me good clud website', ['Sure, here: www.redhat.com', 'ofCourse: you can go to amazon, which works with RedHat website check both']),
    (r'What kinds of clouds are there ?', ['There are three main kinds of clouds: private, public, and hybrid would you like me to explain further? ', 'public, hybrid, private - in secured order']),
    (r'(.*)kinds of clouds(.*)?(.*)', ['There are three main kinds of clouds: private, public, and hybrid would you like me to explain further? ', 'public, hybrid, private - in secured order']),
    (r'please explain further about clouds(.*)', [cloud_explanation, "for the moment I too tired, may be later I will explain. You should've listen in class "]),
    (r'What is tencent(.*)', ["Tencent is a Chinese multinational conglomerate known for its diverse portfolio of internet-based services, including social media, gaming, e-commerce, and more.",
                              "Tencent is a Chinese multinational conglomerate known for its internet-related services, including WeChat, QQ, online gaming, and more. "]),
    (r'(.*)tencent(.*)', ["Tencent is a major player in the global tech industry, with a wide range of services spanning social media, gaming, e-commerce, cloud computing, and more.",
                          "Tencent is one of the largest technology companies in the world, known for its innovation and influence across various internet sectors."]),
    (r'what are some products of tencent(.*)', ["Some of Tencent's key products include WeChat, QQ, Tencent Video, Tencent Music Entertainment, and Tencent Cloud.",
                                              "Tencent offers a diverse range of products, including social networking platforms, online gaming services, and digital entertainment offerings."]),
    (r'what is tencent\s?(\w*)\s?(\w*)\s?(\w*)\s?(\w*)\s?(\w*)?', ["Tencent is renowned for its contribution to various sectors such as social media, gaming, fintech, cloud computing, and artificial intelligence.",
                                                                  "Tencent's influence extends beyond China, with investments and partnerships worldwide in technology, entertainment, and other industries."]),
    (r'how does tencent make money\??', ["Tencent generates revenue primarily through online gaming, advertising, digital content subscriptions, cloud services, and financial technology.",
                                        "Tencent's revenue streams include online gaming, advertising, digital content sales, and investments in other companies."]),
    # You can add more patterns and responses here
]

# Create a chatbot
chatbot = Chat(patterns, reflections)

In [ ]:
from flask import Flask, request, render_template, jsonify
from IPython.display import HTML, display
from firebase import firebase

app = Flask(__name__)

# Global variable to store the input value
arg = None

# Firebase connection
fbConn = firebase.FirebaseApplication('https://braudecloud-18-02-2024-default-rtdb.europe-west1.firebasedatabase.app/', None)

# HTML and JavaScript code for hiding/showing sections
html_code = """







  Cloud Computing Search Engine
























        Welcome to the Cloud Computing Search Engine!







    Search Results






        Interactive Demos

          Demo: Cloud Storage Solutions
          Explore the power of cloud storage with our interactive demo. Learn how to upload, organize, and share your files securely in the cloud. Whether you're a beginner or an advanced user, this demo has something for everyone.



          Demo: Cloud Data Analytics
          Immerse yourself in the world of cloud data analytics with our Demo 2. Discover how to process and analyze large datasets using cloud-based tools. From data visualization to advanced analytics, this demo will showcase the capabilities of cloud data platforms.









        Tutorials

          Tutorial: Getting Started with Cloud Computing
          New to cloud computing? This step-by-step tutorial will guide you through the basics. From understanding key concepts to setting up your first cloud instance, you'll gain the knowledge needed to navigate the cloud confidently. No prior experience required!



          Tutorial: Building Scalable Web Applications in the Cloud
          Ready to take your web development skills to the cloud? This tutorial provides a comprehensive guide on building scalable web applications using cloud services. Learn about serverless architecture, auto-scaling, and best practices for deploying web apps in the cloud.









        Contact Us
        Have a question or feedback? Reach out to us using the form below.



          Your Name:


          Your Email:


          Your Message:









        Chat with us!


          💬








      © 2024 Cloud Computing Search Engine. All rights reserved.
      Privacy Policy
      Contact Us











"""

# Display HTML and JavaScript code
display(HTML(html_code))

# Define a route for handling the root URL
@app.route('/')
def index():
    return render_template('index.html')  # Use a separate HTML file if needed

# Define a route for receiving data from the client
@app.route('/update_arg', methods=['POST'])
def update_arg():
    global arg
    data = request.get_json()
    arg = data['inputValue']
    print('Updated arg:', arg)

    # Call the function and retrieve links
    links = retrieve_links_by_query(arg)
    print(len(links))
    # Send links back to the client
    return jsonify(links)

# Function to retrieve links by query
def retrieve_links_by_query(query):
    # Construct the path to the node containing word occurrences for the query
    query_path = '/index/' + query

    # Retrieve data from Firebase RTDB
    result = fbConn.get(query_path, None)

    if result is not None:
        links = []
        num=5
        # Iterate over each hashed URL
        for hashed_url, data in list(result.items())[:5]:
            # Retrieve the URL from the pages node using the hashed URL
            page_data = fbConn.get('/pages/' + hashed_url, None)

            if page_data:
                links.append(page_data)


        return links
    else:
        return []

@app.route('/chat_message', methods=['POST'])
def chat_message():
    global arg
    data = request.get_json()
    arg = data['inputValue']
    response = chatbot.respond(arg)
    return jsonify(response)


# Run the Flask app
if __name__ == '__main__':
    app.run(port=5000)
